In [197]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import math
import tensorflow as tf
import os
import cv2

from keras import utils
from glob import glob

# http://www.kwangsiklee.com/2018/11/keras%EC%97%90%EC%84%9C-sequence%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC-%EB%8C%80%EC%9A%A9%EB%9F%89-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B-%EC%B2%98%EB%A6%AC%ED%95%98%EA%B8%B0/
# Here, `x_set` is list of path to the images
# and `y_set` are the associated classes.

class G(tf.keras.utils.Sequence):

    def __init__(self, data_path, labels, batch_size= 32, n_channels=3, n_classes=8, shuffle=True, image_size = 256):
        self.data_path, self.labels = data_path, labels# 데이터셋 경로, 데이터 라벨 리스트
        self.batch_size = batch_size # 배치사이즈 설정
        self.n_channels = n_channels # 채널 수
        self.n_classes = n_classes # 클래스 갯수
        self.shuffle = shuffle # 각 epoch마다 새로운 order를 만들어냄
        self.image_size = image_size
        self.on_epoch_end() # 각 에포크의 맨 처음과 끝에 실행됨. 

    def __len__(self):
        return math.ceil(len(self.data_path) / self.batch_size)

    def __getitem__(self, index):
        indexes =  self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        print(indexes)
        # list_IDs_temp = [self.labels[k] for k in indexes]
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.data_path))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)  

    def crop_img (self, img, w, h):
        start = ((w // 2 - 128) , (h // 2 - 128))
        end = ((w // 2 + 128), (h // 2 + 128))
        crop_img = img[start[1]:end[1], start[0]:end[0]]
        return crop_img
            
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.image_size, self.image_size, self.n_channels))
        y = np.empty((self.batch_size), dtype=int) # 메모리만 할당 받고 메모리 초기화는 하지 않는 함수 메모리 초기화를 하려면 zeros를 사용해야함.

        # Generate data
        for i, ID in enumerate(indexes):
            img = cv2.imread(self.data_path[ID])
            h,w,c = img.shape
            crop = self.crop_img(img, w, h)

            X[i, ] = crop
            y[i] = self.labels[ID]
        return X, y
            

In [ ]:
# 확인용 코드
import pandas as pd
from glob import glob

df = pd.read_csv('/Users/jinwook/Downloads/test.csv')

data_path = list(df['data_path'])
print(data_path)
labels = list(df['labels'])
print(labels)

csv_path = 'Landmark/csv/'
csv_lists = glob.glob(csv_path + '*.csv')

def concat_csv(csv_lists):

    for i, csv in enumerate(csv_lists):
        csv_df = pd.read_csv(csv)
        if i == 0:
            df = csv_df
        else:
            df = pd.concat(csv_df)
            
    data_path = list(df['data_path'])
    labels = list(df['labels'])

    g = G(data_path, labels, batch_size=2)
    return g

g = concat_csv(csv_lists)
g.__getitem__(4)